In [3]:
import os
import sys; sys.path.insert(0, os.path.abspath("../../"))
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from dataset.dataset import SkinLesion_Dataset, SegExamples
from pipeline.preprocessing import SkinLesionPreprocessing
from pipeline.feature_extraction import FeaturesExtraction
from pathlib import Path
import time

import logging
import torch
import torchvision

import torch.nn as nn
import torch.optim as optim

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision.models.detection.rpn import AnchorGenerator

In [ ]:
data2 = SkinLesion_Dataset(class_task='binary',resize_image= False)
preproc2 = SkinLesionPreprocessing(resize=True, remove_hair=False)

In [ ]:
datasets = {
        'train': SkinLesion_Dataset(class_task='binary',resize_image= False, partitions=['train']),
        'val': SkinLesion_Dataset(class_task='binary',resize_image= False, partitions=['val'])
        }
def collate_fn(batch):
        return tuple(zip(*batch))
dataloaders = {
        'val': DataLoader(
            datasets['val'], batch_size=5,
            num_workers=4, collate_fn=collate_fn, drop_last=False),
        'train': DataLoader(
            datasets['train'], batch_size=2,
            shuffle=True, collate_fn=collate_fn, num_workers=4, drop_last=False)
    }
